<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week2/2_06_Agent_Basico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AGENTE BASICO

In [1]:
from google.colab import userdata
import os

# Obtener la API key desde userdata
api_key = userdata.get('GOOGLE_API_KEY')

# Opcional: Guardarla como variable de entorno
os.environ['GOOGLE_API_KEY'] = api_key

# Verificar que se haya cargado correctamente
print("API Key cargada:", "Sí" if api_key else "No")
print("Primeros caracteres:", api_key[:10] if api_key else "No encontrada")

API Key cargada: Sí
Primeros caracteres: AIzaSyDx4e


In [2]:
!pip install "langchain>=1,<2" langchain-core langchain-community langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 943.4 kB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.6 MB/s eta 0:00:00


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
print("✅ LangChain-Gemini instalado correctamente")

✅ LangChain-Gemini instalado correctamente


In [5]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

Definir el contexto de ejecución a proporcionar al agente y a las herramientas (tools) acceso a la base de datos.

In [6]:
from dataclasses import dataclass

from langchain_community.utilities import SQLDatabase


# define context structure to support dependency injection
@dataclass
class RuntimeContext:
    db: SQLDatabase

Esta herramienta (tool) se conectará a la base de datos. Observe el uso de `get_runtime` para acceder al contexto de ejecución del grafo.

In [7]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

Agrega un *system prompt* para definir el comportamiento de tus agentes.

In [8]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

¡Crear el agente! Añade un: model, tools, a prompt, y el acceso runtime, and vamos! Puede elegir muchos agentes de la lista de [integracion](https://docs.langchain.com/oss/python/integrations/providers/overview).

In [9]:
import os

In [15]:
# ========================================
# 🤖 Crear modelo Gemini
# ========================================
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # ✅ Modelo válido, estaba con pro
    temperature=0.2,
)

In [16]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,  # ✅ Objeto Gemini (NO cadena)
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)



**PRIMERA CONSULTA**

In [17]:
from langchain_core.messages import HumanMessage

question = "Which are the annual sales per year in  USA??"

inputs = {
    "messages": [HumanMessage(content=question)]
}

for step in agent.stream(
    inputs,
    context=RuntimeContext(db=db),
    stream_mode="values"
):
    msg = step.get("messages", [])[-1]
    msg.pretty_print()

================================ Human Message =================================

Which are the annual sales per year in  USA??
================================== Ai Message ==================================
Tool Calls:
  execute_sql (87c46b02-abea-4fb2-8c7d-6dc4776189ad)
 Call ID: 87c46b02-abea-4fb2-8c7d-6dc4776189ad
  Args:
    query: SELECT STRFTIME('%Y', i.InvoiceDate) AS SaleYear, SUM(il.UnitPrice * il.Quantity) AS AnnualSales FROM customers c JOIN invoices i ON c.CustomerId = i.CustomerId JOIN invoice_items il ON i.InvoiceId = il.InvoiceId WHERE c.Country = 'USA' GROUP BY SaleYear ORDER BY SaleYear;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT STRFTIME('%Y', i.InvoiceDate) AS SaleYear, SUM(il.UnitPrice * il.Quantity) AS AnnualSales FROM customers c JOIN invoices i ON c.CustomerId = i.CustomerId JOIN invoice_items il ON i.InvoiceId = il.InvoiceId WHERE c.

**SEGUNDA CONSULTA**

In [18]:
question = "How many attributes does the album table have??"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

How many attributes does the album table have??
================================== Ai Message ==================================
Tool Calls:
  execute_sql (8c8a456e-f494-4e30-81a9-ec2af0bb7f1a)
 Call ID: 8c8a456e-f494-4e30-81a9-ec2af0bb7f1a
  Args:
    query: PRAGMA table_info(album);
================================= Tool Message =================================
Name: execute_sql

[(0, 'AlbumId', 'INTEGER', 1, None, 1), (1, 'Title', 'NVARCHAR(160)', 1, None, 0), (2, 'ArtistId', 'INTEGER', 1, None, 0)]
================================== Ai Message ==================================

[{'type': 'text', 'text': 'The album table has 3 attributes.', 'extras': {'signature': 'Cs8CAb4+9vu1S6W1TD3u239vIwJIhrHd9fJFVgjNkPT8oymBD11C34patTA4L3amc0C07D7vAwGfRA0dAxVG3T4sWTtS/6vucusR5WiC60Z6wykFwF0HgmUWy0Ur8Ih6yzis9kT4Mvg3v3rynaxzV2f9FP9TEzY0Yb+BBAXJH6ReJ3lH9dqs9IJsUSWPVJu70wvdw5w+DNrWEFAp1ppLdsS/lwRdpd/24iigCZEWdmyMkeS

**TERCERA CONSULTA**

In [19]:
from langchain_core.messages import HumanMessage

question = "Which table has the largest number of entries?"

inputs = {
    "messages": [HumanMessage(content=question)]
}

for step in agent.stream(
    inputs,
    context=RuntimeContext(db=db),
    stream_mode="values"
):
    msg = step.get("messages", [])[-1]
    msg.pretty_print()

================================ Human Message =================================

Which table has the largest number of entries?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (d49e4ffa-e9ab-4c8b-9be2-380ff52d2b4e)
 Call ID: d49e4ffa-e9ab-4c8b-9be2-380ff52d2b4e
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (cd3f735b-23ee-41f5-8655-b9544124bc80)
 Call ID: cd3f735b-23ee-41f5-8655-b9544124bc80
  Args:
    query: SELECT 'Album' AS table_name, COUNT(*) AS row_count FROM Album UNION ALL SELECT 'Artist' AS table_name, COUNT(*) AS row_count FROM Ar

**CUARTA CONSULTA**

In [20]:
from langchain_core.messages import HumanMessage

question = "Obtener los 10 principales artistas por cantidad de ventas. "

inputs = {
    "messages": [HumanMessage(content=question)]
}

for step in agent.stream(
    inputs,
    context=RuntimeContext(db=db),
    stream_mode="values"
):
    msg = step.get("messages", [])[-1]
    msg.pretty_print()

================================ Human Message =================================

Obtener los 10 principales artistas por cantidad de ventas. 
================================== Ai Message ==================================
Tool Calls:
  execute_sql (aca4c584-d334-4f47-a8de-6655fd838a64)
 Call ID: aca4c584-d334-4f47-a8de-6655fd838a64
  Args:
    query: 
SELECT
  a.Name AS ArtistName,
  SUM(il.UnitPrice * il.Quantity) AS TotalSales
FROM artists AS a
JOIN albums AS al
  ON a.ArtistId = al.ArtistId
JOIN tracks AS t
  ON al.AlbumId = t.AlbumId
JOIN invoice_items AS il
  ON t.TrackId = il.TrackId
GROUP BY
  a.Name
ORDER BY
  TotalSales DESC
LIMIT 10;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: artists
[SQL: 
SELECT
  a.Name AS ArtistName,
  SUM(il.UnitPrice * il.Quantity) AS TotalSales
FROM artists AS a
JOIN albums AS al
  ON a.ArtistId = al.ArtistId
JOIN tracks AS t
  ON al.AlbumId = t.

**QUINTA CONSULTA**

In [21]:
question = "Which genre on average has the longest track?"

for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which genre on average has the longest track?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (d8ef5d34-2888-4195-8bd8-23ec620b854c)
 Call ID: d8ef5d34-2888-4195-8bd8-23ec620b854c
  Args:
    query: SELECT g.Name, AVG(t.Milliseconds) AS AverageTrackLength FROM tracks t JOIN genres g ON t.GenreId = g.GenreId GROUP BY g.Name ORDER BY AverageTrackLength DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: tracks
[SQL: SELECT g.Name, AVG(t.Milliseconds) AS AverageTrackLength FROM tracks t JOIN genres g ON t.GenreId = g.GenreId GROUP BY g.Name ORDER BY AverageTrackLength DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execu

**SEXTA CONSULTA**

In [22]:
question = "What are the first 5 artist in the 'Artist' table? execute it"

for step in agent.stream(
    {"messages": question},
    context={"db": db},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What are the first 5 artist in the 'Artist' table? execute it
================================== Ai Message ==================================
Tool Calls:
  execute_sql (bfdd20c2-5eda-412a-978e-7eef91b58c95)
 Call ID: bfdd20c2-5eda-412a-978e-7eef91b58c95
  Args:
    query: SELECT ArtistId, Name FROM Artist LIMIT 5
================================= Tool Message =================================
Name: execute_sql

[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]
================================== Ai Message ==================================

The first 5 artists in the 'Artist' table are:
1. AC/DC
2. Accept
3. Aerosmith
4. Alanis Morissette
5. Alice In Chains
